In [65]:
import os
from shutil import copyfile
import shutil
import numpy as np
import pandas as pd
import fnmatch
import matplotlib.pyplot as plt
from scipy import misc, ndimage
from sklearn.model_selection import train_test_split
import keras
from keras.models import load_model
from keras import backend as K
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Model
from keras.layers import Input
from keras.layers.core import Activation, Reshape
from keras.layers.convolutional import Convolution2D
from keras.layers.normalization import BatchNormalization
import sys
from layers import MaxPoolingWithArgmax2D, MaxUnpooling2D
import matplotlib.pyplot as plt

In [2]:
# print(sys.path)

['C:\\Users\\sshek\\Documents\\My Documents\\Spring 2019 Courses\\Deep Learning and Neural Nets\\Projects\\Project - Final\\Leaf Segmentation_Counting', 'C:\\Users\\sshek\\Documents\\My Documents\\Python Custom Files', 'C:\\Users\\sshek\\Anaconda3\\python37.zip', 'C:\\Users\\sshek\\Anaconda3\\DLLs', 'C:\\Users\\sshek\\Anaconda3\\lib', 'C:\\Users\\sshek\\Anaconda3', '', 'C:\\Users\\sshek\\AppData\\Roaming\\Python\\Python37\\site-packages', 'C:\\Users\\sshek\\Anaconda3\\lib\\site-packages', 'C:\\Users\\sshek\\Anaconda3\\lib\\site-packages\\win32', 'C:\\Users\\sshek\\Anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\Users\\sshek\\Anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\Users\\sshek\\AppData\\Roaming\\Python\\Python37\\site-packages\\IPython\\extensions', 'C:\\Users\\sshek\\.ipython']


In [89]:
df_A1 = pd.read_csv('./Dataset/CVPPP2017_LCC_training/CVPPP2017_LCC_training/training/A1/A1.csv', header = None)
df_A2 = pd.read_csv('./Dataset/CVPPP2017_LCC_training/CVPPP2017_LCC_training/training/A2/A2.csv', header = None)
df_A4 = pd.read_csv('./Dataset/CVPPP2017_LCC_training/CVPPP2017_LCC_training/training/A4/A4.csv', header = None)

In [90]:
columns = ['Plant', 'Count']
plant_df = pd.DataFrame(columns=columns)

In [91]:
## Code for storing Images in ordered format [continous number format] and sorted in separate subdirectories for rgb, fg, center images
baseImPath = './Dataset/CVPPP2017_LCC_training/CVPPP2017_LCC_training/training/'
postImPath = ['A1/','A2/','A4/']
outputPath = './InputDS/CVPPP2017_LCC_training/training2/'
outRgbPath = 'rgb/'
outFgPath = 'fg/'
outCenterPath = 'centers/'

pattern_rgb = "*_rgb.png"
outputRGBPath = outputPath + outRgbPath
count_rgb = 1
pattern_fg = "*_fg.png"
outputFGPath = outputPath + outFgPath
count_fg = 1
pattern_center = "*_centers.png"
outputCenterPath = outputPath + outCenterPath
count_center = 1
for subdir in postImPath:
    inputPath = baseImPath + subdir
    listOfFiles = os.listdir(inputPath) 
    for entry in listOfFiles:  
        if fnmatch.fnmatch(entry, pattern_rgb):
            if subdir == 'A1/':
                plant_df = plant_df.append({'Plant': str(count_rgb) + '.png', 'Count' : df_A1[df_A1[0] == entry][1].item()}, ignore_index = True)
            if subdir == 'A2/':
                plant_df = plant_df.append({'Plant': str(count_rgb) + '.png', 'Count' : df_A2[df_A2[0] == entry][1].item()}, ignore_index = True)
            if subdir == 'A4/':
                plant_df = plant_df.append({'Plant': str(count_rgb) + '.png', 'Count' : df_A4[df_A4[0] == entry][1].item()}, ignore_index = True)
            copyfile(inputPath + entry, outputRGBPath + str(count_rgb) + '.png')
            count_rgb += 1
        if fnmatch.fnmatch(entry, pattern_fg):
            copyfile(inputPath + entry, outputFGPath + str(count_fg) + '.png')
            count_fg += 1
        if fnmatch.fnmatch(entry, pattern_center):
            copyfile(inputPath + entry, outputCenterPath + str(count_center) + '.png')
            count_center += 1

In [94]:
plant_df.to_csv('./InputDS/CVPPP2017_LCC_training/PlantCounts.csv', index=False)

In [95]:
## Since there is no additional validation and test data, removing 50 validation and 50 test images from the training set on a random basis
## Separating images into training, validation and test sets
x = np.arange(1,784,1)
X_train, X_val_test = train_test_split(x, test_size=0.1277, random_state=1)
X_val, X_test = train_test_split(X_val_test, test_size=0.50, random_state=1)
print(len(X_train))
print(len(X_val))
print(len(X_test))
print(X_train)
print(X_val)
print(X_test)

683
50
50
[217 712 269 202 287 449 190 626 482  62 693 324 525 602   4 552 364 658
 275  66 409 176 182 105 390 742  58 315 429 577 739 428 393 762 369 188
 351  36 422 513  42 389 724 580 109 781 571   1 613  75 335  67 258 512
 474 359 162 611 579 529 342 353 682  14 575 403 346 278  93 126 115 683
 779 557 468 772 332 133 517 544 774 494 475 518  41 503 765  69  77 181
 780 365 140 443 647 119 745 196 721 605 497  35 263 759 638 776 161 728
 603  15 248 120  83 386 736 239 108  18 465 173 719 354 104 440 645 412
 395 473  46 651 676 371 249 448 662 564  12 215  89 640 561 148 556 300
 256 383 312 621 778 355 372 675 174 653 122 760 511  32 311 594 431 286
 714 117 274  96 488 387 306 330 299 532  63 460 539 507  92 737 362 635
  30  55 732 373 147 612 624 601 710 166 415  81 433 260  10 677 614 743
 180 502 692 294 376   6 198 715 615 483 262 350 718 480  39 672 356 747
 163  68 125  40   5  34 618 281 545 100 660 591 237 400 453 709 367 588
 756 519 434  13 187 487 536 713 261 375 

In [96]:
columns = ['Plant', 'Count']
train_df = pd.DataFrame(columns=columns)
val_df = pd.DataFrame(columns=columns)
test_df = pd.DataFrame(columns=columns)

In [98]:
### Paths for input from train and into Validation folder
inTrainRgbPath = './InputDS/CVPPP2017_LCC_training/training2/rgb/'
outValRgbPath = './InputDS/CVPPP2017_LCC_training/validation2/rgb/'
inTrainFgPath = './InputDS/CVPPP2017_LCC_training/training2/fg/'
outValFgPath = './InputDS/CVPPP2017_LCC_training/validation2/fg/'
inTrainCentersPath = './InputDS/CVPPP2017_LCC_training/training2/centers/'
outValCentersPath = './InputDS/CVPPP2017_LCC_training/validation2/centers/'
### Paths for input from train and into Test folder
inTrainRgbPath = './InputDS/CVPPP2017_LCC_training/training2/rgb/'
outTestRgbPath = './InputDS/CVPPP2017_LCC_training/testing2/rgb/'
inTrainFgPath = './InputDS/CVPPP2017_LCC_training/training2/fg/'
outTestFgPath = './InputDS/CVPPP2017_LCC_training/testing2/fg/'
inTrainCentersPath = './InputDS/CVPPP2017_LCC_training/training2/centers/'
outTestCentersPath = './InputDS/CVPPP2017_LCC_training/testing2/centers/'
### Moving the training images to new path and renumbering then
inTrainRgbPath = './InputDS/CVPPP2017_LCC_training/training2/rgb/'
outTrainRgbPath = './InputDS/CVPPP2017_LCC_training/train2/rgb/'
inTrainFgPath = './InputDS/CVPPP2017_LCC_training/training2/fg/'
outTrainFgPath = './InputDS/CVPPP2017_LCC_training/train2/fg/'
inTrainCentersPath = './InputDS/CVPPP2017_LCC_training/training2/centers/'
outTrainCentersPath = './InputDS/CVPPP2017_LCC_training/train2/centers/'

## Copying and arranging validation folder
counter = 1
for val in X_val:
    shutil.move(inTrainRgbPath + str(val) + '.png', outValRgbPath + str(counter) + '.png')
    shutil.move(inTrainFgPath + str(val) + '.png', outValFgPath + str(counter) + '.png')
    shutil.move(inTrainCentersPath + str(val) + '.png', outValCentersPath + str(counter) + '.png')
    val_df = val_df.append({'Plant': str(counter) + '.png', 'Count' : plant_df[plant_df['Plant'] == str(val) + '.png']['Count'].item()}, ignore_index = True)
    counter += 1
    
counter = 1
## Copying and arranging test folder
for test in X_test:
    shutil.move(inTrainRgbPath + str(test) + '.png', outTestRgbPath + str(counter) + '.png')
    shutil.move(inTrainFgPath + str(test) + '.png', outTestFgPath + str(counter) + '.png')
    shutil.move(inTrainCentersPath + str(test) + '.png', outTestCentersPath + str(counter) + '.png')
    test_df = test_df.append({'Plant': str(counter) + '.png', 'Count' : plant_df[plant_df['Plant'] == str(test) + '.png']['Count'].item()}, ignore_index = True)
    counter += 1
    
## Renumbering the remaining 683 entries in training data starting from 1
counter = 1
for train in X_train:
    shutil.move(inTrainRgbPath + str(train) + '.png', outTrainRgbPath + str(counter) + '.png')
    shutil.move(inTrainFgPath + str(train) + '.png', outTrainFgPath + str(counter) + '.png')
    shutil.move(inTrainCentersPath + str(train) + '.png', outTrainCentersPath + str(counter) + '.png')
    train_df = train_df.append({'Plant': str(counter) + '.png', 'Count' : plant_df[plant_df['Plant'] == str(train) + '.png']['Count'].item()}, ignore_index = True)
    counter += 1

In [99]:
train_df.to_csv('./InputDS/CVPPP2017_LCC_training/Train_PlantCounts.csv', index=False)
val_df.to_csv('./InputDS/CVPPP2017_LCC_training/Val_PlantCounts.csv', index=False)
test_df.to_csv('./InputDS/CVPPP2017_LCC_training/Test_PlantCounts.csv', index=False)

In [151]:
# Applying Data Augumentation to each RGB image and its mask and storing it
inPath = './InputDS/CVPPP2017_LCC_training/train/rgb/'
outPath = './InputDS/CVPPP2017_LCC_training/train/transformedImgs/'
listOfFiles = os.listdir(inPath) 
counter = 1
datagen = ImageDataGenerator(rotation_range = 15, width_shift_range = 0.1, height_shift_range = 0.1, shear_range = 0.15, zoom_range = 0.1, channel_shift_range = 10., horizontal_flip = True, vertical_flip = True)
for entry in listOfFiles: 
    inputPath = inPath + entry
    outputPath = outPath + str(counter) + '/'
    if not os.path.exists(outputPath):
        os.makedirs(outputPath)
    counter += 1
    i = 1
    rgb_image = image.load_img(inputPath, target_size=(224, 224))
    x = img_to_array(rgb_image)  
    x = x.reshape((1,) + x.shape)  
    for batch in datagen.flow(x, batch_size=1, save_to_dir=outputPath, save_format='png'):
        i += 1
        if i > 100:
            break

In [163]:
# Now transfer all the original images with their respective augumented images. This creates 101 images in total for each original rgb image
inPath = './InputDS/CVPPP2017_LCC_training/train/rgb/'
outPath = './InputDS/CVPPP2017_LCC_training/train/transformedImgs/'
listOfFiles = os.listdir(inPath) 
counter = 1
for entry in listOfFiles:
        img = image.load_img(inPath + entry, target_size=(224, 224))
        img_array = image.img_to_array(img)
        image.save_img(outPath + str(counter) + '/' + str(counter) + '_original.png', img_array)
        counter += 1
print(counter)

684


In [44]:
# Convert all images to (224,224)
# targetPath = './InputDS/CVPPP2017_LCC_training/train/transformedImgs/1/_0_37.png'
# targetPath = './InputDS/CVPPP2017_LCC_training/train/trial/1_originalImg.png'
targetPath = './InputDS/CVPPP2017_LCC_training/train2/rgb'
img = load_img(targetPath)
x = img_to_array(img) 
print(x.shape)
# Experimental Section Ends

(441, 441, 3)


In [105]:
## Change all the train labels to the same size as the train images
inputPath = './InputDS/CVPPP2017_LCC_training/testing2/fg/'
targetPath = './InputDS/CVPPP2017_LCC_training/final_test/test_masks/'
listOfFiles = os.listdir(inputPath) 
for entry in listOfFiles:   
    img = image.load_img(inputPath + entry, target_size=(224, 224))
    img_array = image.img_to_array(img)
    image.save_img(targetPath + entry, img_array)

In [14]:
inputPath = './InputDS/CVPPP2017_LCC_training/final/val_frames/val/'
# targetPath = './InputDS/CVPPP2017_LCC_training/final/val_masks/val2/'
listOfFiles = os.listdir(inputPath) 
for entry in listOfFiles:   
    img = image.load_img(inputPath + entry)
    img_array = image.img_to_array(img)
#     image.save_img(targetPath + entry, img_array)
#     print(inputPath + entry)
    print("Image: " + str(img_array.shape))

Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 224, 3)
Image: (224, 